# Python для анализа данных

## Веб-скрэйпинг: переход по ссылкам, скачивание файлов

*Автор: Татьяна Рогович, НИУ ВШЭ*

На прошлом занятии мы с вами работали с табличными данными. Второй очень частой задачей для скрэйпинга является автоматический переход по ссылкам. Обычно мы встречаемся с двумя сценариями: переход по нумерованным страницам (обычно это выдача поиска или некоторый упорядоченный архив документов) и переход по определенным ссылкам на странице. Сегодня будем учиться делать и то, и то.

Давайте начнем с учебного примера.

# Задача

По ссылке http://py4e-data.dr-chuck.net/known_by_Lilian.html список людей, которых знает Лилиан. Нужно найти ее 18-го друга (считаем с одного) и перейти по ссылке (там будет список людей, которых знает этот друг). Какой имя вы извлечете, если эту операцию повторить 7 раз? То есть нам нужно найти 18-го друга 6-го человека.

Прежде всего изучите исходный код страницы. В каком теге лежат ссылки?

In [1]:
# решение
import requests 
from bs4 import BeautifulSoup

friends = requests.get('http://py4e-data.dr-chuck.net/known_by_Lilian.html').text
soup = BeautifulSoup(friends, 'lxml')
print(soup.find_all('a')[17]) # ссылки лежат в тэге 'a', находим 18-го друга

<a href="http://py4e-data.dr-chuck.net/known_by_Tigan.html">Tigan</a>


Обратите внимание, что нужная нам информация лежит в атрибуте href, достать текст, как мы делали раньше не поможет.

In [2]:
print(soup.find_all('a')[17].get('href')) # с помощью метода get достаем информацию из атрибут href

http://py4e-data.dr-chuck.net/known_by_Tigan.html


Теперь осталось упаковать все в цикл.

In [3]:
friends = requests.get('http://py4e-data.dr-chuck.net/known_by_Lilian.html').text
soup = BeautifulSoup(friends, 'lxml')

for ix in range(6):
    link = soup.find_all('a')[17].get('href')
    print(link)
    soup = BeautifulSoup(requests.get(link).text, 'lxml')
    
print('Ответ: '+soup.find_all('a')[17].text)

http://py4e-data.dr-chuck.net/known_by_Tigan.html
http://py4e-data.dr-chuck.net/known_by_Mickey.html
http://py4e-data.dr-chuck.net/known_by_Marvin.html
http://py4e-data.dr-chuck.net/known_by_Yago.html
http://py4e-data.dr-chuck.net/known_by_Daood.html
http://py4e-data.dr-chuck.net/known_by_Jillian.html
Ответ: Bradly


## Скачивание файлов

Кстати, еще одно применение скрэйпинга, о котором мы пока не поговорили - это автоматизированное скачивание файлов. Например, вы хотите скачать научные статьи по определенному ключевому слову или прайс-листы поставщика, которые он загружает на сайт в эскеле.

Давайте посмотрим, как скачивать файлы на примере pdf и заодно попробуем походить по ссылкам. Кстати, этот процесс еще часто называется spidering или crawling, потому что ваш скрипт как паучок ползет по ссылкам (отсюда и название первых поисковых роботов - spider).

Давайте попробуем сохранить англоязычные summary дисертаций, защищенных в 2019 году

Мы уже отредактировали фильтры и ссылка их запомнила. Позже сегодня посмотрим как можно самим заполнять такие поля с помощью Selenium.

https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes

In [4]:
link = 'https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes'
soup = BeautifulSoup(requests.get(link).text, 'lxml')

Давайте для начала поэкспериментируем с первым кандидатом.

In [5]:
print(len(soup.find_all('a', {'class':'link'})))
soup.find_all('a', {'class':'link'})[:10]

175


[<a class="link" href="https://www.hse.ru/cookie.html" target="_blank" title="Пройти по ссылке">здесь</a>,
 <a class="link" href="https://www.hse.ru/data_protection_regulation" target="_blank" title="Пройти по ссылке">здесь</a>,
 <a class="link no-visited with-icon with-icon_flag-spb" href="//spb.hse.ru/">Санкт-Петербург</a>,
 <a class="link no-visited with-icon with-icon_flag-nn" href="//nnov.hse.ru/">Нижний Новгород</a>,
 <a class="link no-visited with-icon with-icon_flag-perm" href="https://perm.hse.ru/">Пермь</a>,
 <a class="link link_no-visited link_no-underline" href="/en/">EN</a>,
 <a class="link link_dark no-visited" href="//www.hse.ru/search/search.html?simple=0&amp;searchid=2284688">Расширенный поиск по сайту</a>,
 <a class="link no-visited header_breadcrumb__link" href="//www.hse.ru"><span>Национальный исследовательский университет «Высшая школа экономики»</span></a>,
 <a class="link" href="/sci/diss/index.html?spec=&amp;chief=&amp;fulltext=yes&amp;author=&amp;council=&amp;d

Видим, что ссылок очень много, а нам нужны только те, которые ведут на summary. Можно поискать их по тексту ссылки.

In [6]:
for link in soup.find_all('a', text='Summary'):
    print(link)

<a class="link" data-hse-file="PDF" href="/data/2019/10/29/1532361365/summary_EN_final%20(3).pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/22/1528885739/Mitrofanova%20-%20summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/18/1543364021/Borodina%20E._Summary_181019.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/16/1239735382/%D0%9C%D0%B5%D0%BB%D0%B8%D0%BA%D1%8F%D0%BD_Summary%2016.10.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/03/1182240389/%D0%9A%D1%83%D1%87%D0%B8%D0%BD_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_ENG.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/15/1539883570/summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/11/1541034023/Resume.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/11/1535827784/kulikova_thesis%20summary_final.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/

И соберем ссылки в список.

In [7]:
links = []
for link in soup.find_all('a', text='Summary'):
    print(link)
    links.append(link.get('href'))
    
print(links)

<a class="link" data-hse-file="PDF" href="/data/2019/10/29/1532361365/summary_EN_final%20(3).pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/22/1528885739/Mitrofanova%20-%20summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/18/1543364021/Borodina%20E._Summary_181019.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/16/1239735382/%D0%9C%D0%B5%D0%BB%D0%B8%D0%BA%D1%8F%D0%BD_Summary%2016.10.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/03/1182240389/%D0%9A%D1%83%D1%87%D0%B8%D0%BD_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_ENG.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/15/1539883570/summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/11/1541034023/Resume.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/2019/10/11/1535827784/kulikova_thesis%20summary_final.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="/data/

Отлично. Теперь нужно придумать, откуда взять название для каждого файла. Пусть это будут фамилии авторов, давайте доберемся до них. Такую задачу мы пока не решали: будем искать тэг по тексту, а потом искать его родителя (потому что ни ячейку таблицы с именем автора, ни саму таблицу не получится уникально отсечь).

In [8]:
for author in soup.find_all('td', text='Соискатель:'):
    print(author.parent())

[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v">Белобородова Полина Михайловна</td>]
[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v"><a class="link" href="/staff/emitrofanova">Митрофанова Екатерина Сергеевна</a></td>, <a class="link" href="/staff/emitrofanova">Митрофанова Екатерина Сергеевна</a>]
[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v">Бородина Елена Николаевна</td>]
[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v"><a class="link" href="/org/persons/205653">Меликян Алиса Валерьевна</a></td>, <a class="link" href="/org/persons/205653">Меликян Алиса Валерьевна</a>]
[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v"><a class="link" href="/org/persons/65825225">Кучин Илья Игоревич</a></td>, <a class="link" href="/org/persons/65825225">Кучин Илья Игоревич</a>]
[<td class="p0 p" style="width: 200px;">Соискатель:</td>, <td class="p1 v"><a class="link" href

Достанем фамилии.

In [9]:
authors = []
for author in soup.find_all('td', text='Соискатель:'):
    print(author.parent()[1].get_text().split()[0])
    authors.append(author.parent()[1].get_text().split()[0])
    
print(authors)

Белобородова
Митрофанова
Бородина
Меликян
Кучин
Монахов
Годин
Куликова
Сидорова
Захарова
['Белобородова', 'Митрофанова', 'Бородина', 'Меликян', 'Кучин', 'Монахов', 'Годин', 'Куликова', 'Сидорова', 'Захарова']


Проверим, что списки действительно одинаковой длины.

In [10]:
len(links) == len(authors)

True

Теперь давайте попробуем реализовать сохранение файла для одного автора, а потом соберем все это в цикл. Для начала "починим" ссылки, добавив в начало адрес домена.

In [11]:
links = ['https://www.hse.ru' + link for link in links]
print(links)

['https://www.hse.ru/data/2019/10/29/1532361365/summary_EN_final%20(3).pdf', 'https://www.hse.ru/data/2019/10/22/1528885739/Mitrofanova%20-%20summary.pdf', 'https://www.hse.ru/data/2019/10/18/1543364021/Borodina%20E._Summary_181019.pdf', 'https://www.hse.ru/data/2019/10/16/1239735382/%D0%9C%D0%B5%D0%BB%D0%B8%D0%BA%D1%8F%D0%BD_Summary%2016.10.pdf', 'https://www.hse.ru/data/2019/10/03/1182240389/%D0%9A%D1%83%D1%87%D0%B8%D0%BD_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_ENG.pdf', 'https://www.hse.ru/data/2019/10/15/1539883570/summary.pdf', 'https://www.hse.ru/data/2019/10/11/1541034023/Resume.pdf', 'https://www.hse.ru/data/2019/10/11/1535827784/kulikova_thesis%20summary_final.pdf', 'https://www.hse.ru/data/2019/08/26/1536057698/%D0%A1%D0%B8%D0%B4%D0%BE%D1%80%D0%BE%D0%B2%D0%B0_summary.pdf', 'https://www.hse.ru/data/2019/10/04/1184372726/%D0%A0%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%BD%D0%B0%20%D0%B0%D0%BD%D0%B3%D0%BB%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%BC_%D0%97%D0%B0%D1%85%D0%B0%D1%80%D0%BE%D0%B2%D0%B

Теперь попробуем сохранить файл. У нас все файлы pdf, будем переименовывать их фамилиями автора. Кстати, если файлы разного формата, то расширение можно узнать через атрибут headers

In [12]:
requests.get(links[0]).headers['content-type']

'application/pdf'

In [15]:
summary = requests.get(links[0], stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
if summary.headers['content-type'] == 'application/pdf': # на всякий случай делаем проверку, иначе получим битый файл
    fh = open('test.pdf', 'wb') # wb - запись байтовой информации
    fh.write(summary.content) # считываем туда "содержание" файла по ссылке
    fh.close()

Давайте теперь еще добавим имя файла по фамилии.

In [16]:
summary = requests.get(links[0], stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
if summary.headers['content-type'] == 'application/pdf': # на всякий случай делаем проверку, иначе получим битый файл
    fh = open(f'{authors[0]}.pdf', 'wb') # wb - запись байтовой информации
    fh.write(summary.content) # считываем туда "содержание" файла по ссылке
    fh.close()

Давайте сохраним обработку файла в функцию и соберем уже все в цикл.

In [17]:
def get_pdf(idx):
    summary = requests.get(links[idx], stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
    if summary.headers['content-type'] == 'application/pdf': # на всякий случай делаем проверку, иначе получим битый файл
        fh = open(f'{authors[idx]}.pdf', 'wb') # wb - запись байтовой информации
        fh.write(summary.content) # считываем туда "содержание" файла по ссылке
        fh.close()

In [18]:
link = 'https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes'
soup = BeautifulSoup(requests.get(link).text, 'lxml')

links = []
for link in soup.find_all('a', text='Summary'):
    links.append('https://www.hse.ru' + link.get('href'))

authors = []
for author in soup.find_all('td', text='Соискатель:'):
    authors.append(author.parent()[1].get_text().split()[0])
    
for idx in range(len(authors)):
    get_pdf(idx)

Получилось! И последнее. У нас было несколько страниц, давайте по страницам тоже пройдемся. Кликните в браузере на страницу 2 по ссылке, а потом обратно на 1. Обратите внимание, что ссылка в браузере поменялась и теперь в ней появился параметр index. Скопируем новую ссылку для первой страницы.

In [19]:
page = 1 # создаю переменную и с помощью форматирования строк добавляю ее в ссылку после index
link = f'https://www.hse.ru/sci/diss/index{page}.html?spec=&chief=&fulltext=yes&author=&council=&degree_type=&type=1&year=2019'

In [20]:
link # проверили, что работает

'https://www.hse.ru/sci/diss/index1.html?spec=&chief=&fulltext=yes&author=&council=&degree_type=&type=1&year=2019'

In [21]:
for page in range(1,7): 
    link = f'https://www.hse.ru/sci/diss/index{page}.html?spec=&chief=&fulltext=yes&author=&council=&degree_type=&type=1&year=2019'
    soup = BeautifulSoup(requests.get(link).text, 'lxml')

    links = []
    for link in soup.find_all('a', text='Summary'):
        links.append('https://www.hse.ru' + link.get('href'))

    authors = []
    for author in soup.find_all('td', text='Соискатель:'):
        authors.append(author.parent()[1].get_text().split()[0])

    for idx in range(len(authors)):
        get_pdf(idx)

Готово!